# Instruction Finetuning

## 데이터셋 구축
1. 목적 정의: 먼저, 세부 튜닝을 통해 달성하고자 하는 목표를 명확히 합니다.

2. 데이터 수집: 목표에 맞는 데이터를 수집합니다. 이 데이터는 공개 데이터셋일 수도 있고, 사용자가 직접 수집한 데이터일 수도 있습니다.

3. 데이터 가공: 수집한 데이터를 모델 훈련에 적합하게 가공합니다. 이 과정에서는 데이터를 정제하고, 필요한 형식으로 변환하는 작업이 포함됩니다.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 1번 GPU만 노출됨

### 공개 데이터셋 다운로드

In [3]:
from datasets import Dataset, load_dataset

In [4]:
dataset = load_dataset("royboy0416/ko-alpaca")

In [5]:
# 데이터 구조 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 49620
    })
})


In [6]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}

## Gemma 데이터셋 포맷팅

```
<start_of_turn>user
What is Cramer's Rule?<end_of_turn>
<start_of_turn>model
Cramer's Rule is ...<end_of_turn>

In [7]:
# 'prompt' 필드 생성 함수
def format_instruction(example):

    # 추가 컨텍스트(input 필드)가 있는 경우
    if example['input'] and len(example['input']) > 0:
        text = f"""<start_of_turn>user\n{example["instruction"]}\n{example["input"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""
    # input 필드가 없는 경우
    else:
        text = f"""<start_of_turn>user\n{example["instruction"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""

    return {'prompt': text}

# 데이터셋의 prompt 필드를 업데이트
dataset = dataset.map(format_instruction)

In [8]:
dataset['train'][0]

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
 'input': '',
 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n건강을 유지하기 위한 세 가지 팁을 알려주세요.\n\n### Response:\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.',
 'prompt': '<start_of_turn>user\n건강을 유지하기 위한 세 가지 팁을 알려주세요.<end_of_turn>\n<start_of_turn>model\n세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.<end_of_turn>'}

In [9]:
dataset['train'][5]

{'instruction': '홀수 중 하나를 밝히세요.',
 'input': '트위터, 인스타그램, 텔레그램',
 'output': '텔레그램입니다.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n홀수 중 하나를 밝히세요.\n\n### Input:\n트위터, 인스타그램, 텔레그램\n\n### Response:\n텔레그램입니다.',
 'prompt': '<start_of_turn>user\n홀수 중 하나를 밝히세요.\n트위터, 인스타그램, 텔레그램<end_of_turn>\n<start_of_turn>model\n텔레그램입니다.<end_of_turn>'}

## 모델 로드 및 튜닝:
1. 모델 학습: gemma-2b 모델을 로드하고, 준비된 데이터셋을 사용하여 모델을 세부 튜닝합니다. 이 과정에서는 학습률, 에폭 수 등의 파라미터를 조정할 수 있습니다.
2. 평가 및 반복: 튜닝된 모델을 평가하고 결과를 확인합니다. 필요에 따라 여러 번 반복하여 모델의 성능을 최적화할 수 있습니다.

!pip install -qU transformers==4.38.0 accelerate==0.27.1 bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10

In [10]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

In [11]:
model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 39696
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 9924
    })
})

In [14]:
train_data = dataset["train"]
test_data = dataset["test"]

In [15]:
print(train_data[0])

{'instruction': '도덕적 교훈이 담긴 짧은 이야기 구성하기', 'input': '', 'output': '한 번에 두 개의 문을 열면서 그렇게 하면 안 된다는 교훈을 담은 짧은 이야기는 "한 번에 다 먹고 싶어한다" 이다.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request. \n\n### Instruction:\n도덕적 교훈이 담긴 짧은 이야기 구성하기\n\n### Response:\n한 번에 두 개의 문을 열면서 그렇게 하면 안 된다는 교훈을 담은 짧은 이야기는 "한 번에 다 먹고 싶어한다" 이다.', 'prompt': '<start_of_turn>user\n도덕적 교훈이 담긴 짧은 이야기 구성하기<end_of_turn>\n<start_of_turn>model\n한 번에 두 개의 문을 열면서 그렇게 하면 안 된다는 교훈을 담은 짧은 이야기는 "한 번에 다 먹고 싶어한다" 이다.<end_of_turn>', 'input_ids': [2, 106, 1645, 108, 236840, 241618, 237603, 75985, 241573, 235832, 205799, 241615, 235248, 245115, 236648, 202926, 236386, 220641, 72159, 107, 108, 106, 2516, 108, 236511, 84389, 236179, 93828, 49532, 236137, 45980, 236392, 120699, 237722, 236554, 20350, 241838, 237458, 30181, 237722, 70685, 95622, 161509, 75985, 241573, 236392, 205799, 236648, 235248, 245115, 236648, 202926, 236386, 236214, 664, 236511, 84389, 236179, 32048, 22244

In [16]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """<start_of_turn>user
  {query}
  <end_of_turn>
  <start_of_turn>model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

# Fine tuning 이전
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.", model=model, tokenizer=tokenizer)
print(result)

user
  건강을 유지하기 위한 세 가지 팁을 알려주세요.
  
  model
  
  abbaye
  이것은 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 궁극적으로 


In [17]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Format your dataset to have "input_ids" and "labels" directly
# This avoids the need for dataset_text_field parameter
def preprocess_function(examples):
    # Assuming examples has a "prompt" field
    inputs = tokenizer(examples["prompt"], padding="max_length", truncation=True)
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": inputs["input_ids"].copy()
    }

# Apply preprocessing to your datasets
processed_train_data = train_data.map(preprocess_function)
processed_test_data = test_data.map(preprocess_function)

trainer = SFTTrainer(
    model=model,
    train_dataset=processed_train_data,
    eval_dataset=processed_test_data,
    # Remove the problematic parameter entirely
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

Map:   0%|          | 0/39696 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/9924 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Converting train dataset to ChatML:   0%|          | 0/39696 [00:00<?, ? examples/s]

...

Applying chat template to eval dataset:   0%|          | 0/9924 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/9924 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.662400
20,2.905700
30,2.548000
40,2.461300
50,2.287900
60,2.209900
70,2.127300
80,2.058200
90,2.239200
100,2.153600


TrainOutput(global_step=100, training_loss=2.46533447265625, metrics={'train_runtime': 26.3067, 'train_samples_per_second': 15.205, 'train_steps_per_second': 3.801, 'total_flos': 394225143656448.0, 'train_loss': 2.46533447265625})

In [ ]:
# def get_completion(query, model, tokenizer, max_new_tokens=512):
#     # 질문-답변 형식으로 프롬프트 구성
#     formatted_query = f"### 질문: {query}\n\n### 답변:"
    
#     inputs = tokenizer(formatted_query, return_tensors="pt").to(model.device)
    
#     with torch.no_grad():
#         outputs = model.generate(
#             input_ids=inputs["input_ids"],
#             attention_mask=inputs["attention_mask"],
#             max_new_tokens=max_new_tokens,
#             do_sample=True,
#             temperature=0.7,
#             top_p=0.9,
#             repetition_penalty=1.5,  # 반복 패널티 증가
#             no_repeat_ngram_size=3,  # 3-그램 반복 방지
#             pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
#         )
    
#     response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
#     return response

In [ ]:
# def get_completion(query, model, tokenizer, max_new_tokens=512):
#     # 추가 지시사항: 코드, 추가 설명, 혹은 다른 형식의 텍스트 없이
#     # 순수하게 7개의 건강 팁만을 나열하도록 요청합니다.
#     # 각 항목은 '1 -', '2 -'와 같이 번호를 붙여 간결하게 작성해야 합니다.
#     additional_instructions = (
#         "코드, 추가 설명, 혹은 다른 형식의 텍스트는 포함하지 말고, "
#         "각 항목은 '1 -', '2 -'와 같이 번호를 붙여서 나열해 주시고, "
#         "각 항목은 간결하고 명확하게 작성해 주세요."
#     )

#     # 질문과 추가 지시사항을 함께 프롬프트 형식으로 구성합니다.
#     # "### 질문:"과 "### 답변:" 구분자를 사용하여 모델이 질문과 답변을 명확하게 인식하도록 합니다.
#     formatted_query = f"### 질문: {query}\n{additional_instructions}\n\n### 답변:"

#     # 토크나이저를 사용하여 입력 텍스트를 토큰화하고, 모델의 장치(device)로 보냅니다.
#     inputs = tokenizer(formatted_query, return_tensors="pt").to(model.device)

#     # 모델을 평가 모드(no_grad)로 설정한 후, 지정한 파라미터로 텍스트를 생성합니다.
#     with torch.no_grad():
#         outputs = model.generate(
#             input_ids=inputs["input_ids"],              # 입력 텍스트 토큰
#             attention_mask=inputs["attention_mask"],      # 주의 마스크
#             max_new_tokens=max_new_tokens,                # 생성할 최대 토큰 수
#             do_sample=True,                               # 샘플링 사용
#             temperature=0.7,                              # 생성 텍스트의 다양성 조절
#             top_p=0.8,                                    # 누적 확률 분포의 임계값
#             repetition_penalty=1.2,                       # 반복되는 구문 방지
#             no_repeat_ngram_size=3,                       # 3-그램 이상 반복 방지
#             pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id  # 패딩 토큰 지정
#         )

#     # 프롬프트 부분을 제외한 생성된 텍스트만 추출합니다.
#     response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
#     return response

In [ ]:
import re
import torch

def extract_expected_count(query):
    """
    질문 내에서 숫자(항목 수)를 추출합니다.
    예를 들어, "일곱 가지", "7개" 등의 표현에서 7을 추출할 수 있습니다.
    현재는 아라비아 숫자만 추출하지만, 필요에 따라 한글 숫자도 추가할 수 있습니다.
    """
    match = re.search(r'(\d+)', query)
    if match:
        return int(match.group(1))
    return None

def extract_expected_length(query):
    """
    질문 내에서 '글자 수'와 관련된 요구사항을 추출하는 예시 함수입니다.
    "최소 100글자", "200자 이상" 등의 표현을 인식할 수 있도록 정규표현식을 확장할 수 있습니다.
    현재는 간단한 예제로, 추출된 경우 정수값을 반환합니다.
    """
    match = re.search(r'(\d+)\s*(글자|자)', query)
    if match:
        return int(match.group(1))
    return None

def get_completion(query, model, tokenizer, max_new_tokens=512):
    """
    주어진 질문에 대해 모델이 스스로 질문을 분석하고,
    주제, 요구되는 항목 수, 글자 수 등의 조건에 맞는 답변을 생성하도록 합니다.
    
    :param query: 사용자 질문 (예: "건강을 유지하기 위한 일곱 가지 팁을 알려주세요.")
    :param model: 사전 학습된 모델
    :param tokenizer: 모델에 맞는 토크나이저
    :param max_new_tokens: 생성할 최대 토큰 수
    :return: 모델이 생성한 응답 텍스트
    """
    # 1. 질문에서 요청된 항목 수와 글자 수를 추출합니다.
    expected_count = extract_expected_count(query)
    expected_length = extract_expected_length(query)
    
    # 2. 메타 프롬프트: 모델이 질문의 주제와 요구사항(항목 수, 글자 수 등)을 스스로 분석하도록 유도합니다.
    meta_instructions = (
        "질문의 주제와 요구사항(예: 항목 수, 글자 수 등)을 정확히 파악한 후, "
        "질문에 부합하는 답변을 생성해 주세요. "
    )
    
    # 3. 기본 지시사항: 코드, 추가 설명, 혹은 다른 형식의 텍스트 없이, 순수하게 답변 내용만을 출력하도록 요청합니다.
    base_instructions = (
        "코드, 추가 설명, 혹은 다른 형식의 텍스트는 포함하지 말고, "
        "질문에 맞는 답변을 순수하게 출력해 주세요. "
    )
    
    # 4. 만약 expected_count가 추출되었다면, 해당 항목 수에 맞게 출력하라는 조건을 추가합니다.
    if expected_count is not None:
        count_instruction = f"출력은 반드시 {expected_count}개의 항목으로 구성되어야 합니다. "
    else:
        count_instruction = ""
    
    # 5. 만약 expected_length가 추출되었다면, 답변의 총 글자 수가 해당 값 이상이 되도록 조건을 추가합니다.
    if expected_length is not None:
        length_instruction = f"답변의 총 글자 수는 최소 {expected_length}자 이상이어야 합니다. "
    else:
        length_instruction = ""
    
    # 6. 모든 지시사항을 결합하여 최종 추가 지시사항을 생성합니다.
    additional_instructions = meta_instructions + count_instruction + length_instruction + base_instructions
    
    # 7. "### 질문:"과 "### 답변:" 구분자를 사용하여 프롬프트를 구성합니다.
    formatted_query = f"### 질문: {query}\n{additional_instructions}\n\n### 답변:"
    
    # 8. 입력 텍스트를 토크나이저로 토큰화하고 모델이 위치한 디바이스로 옮깁니다.
    inputs = tokenizer(formatted_query, return_tensors="pt").to(model.device)
    
    # 9. 모델을 평가 모드(no_grad)로 설정한 후, 지정한 파라미터로 텍스트를 생성합니다.
    #    do_sample=True로 하여 약간의 무작위성을 유지하면서도 메타 지시사항에 따라 응답이 생성되도록 합니다.
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.8,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
        )
    
    # 10. 입력 프롬프트 부분을 제외한 생성된 텍스트만 디코딩합니다.
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    return response


1. 기본적인 건강관리 방법으로 체중 관리, 규칙적으로 운동 및 식습관 관리, 스트레스 관리, 우울증 관리를 들 수 있습니다.
2. 흡연과 음주를 줄이고 과잉노력을 줄이는 것이 중요합니다.
3. 우울감을 완화시키기 위해서는 외부에서 도움을 받을 수 있도록 하는 것도 중요합니다
4. 이러한 일곱가지 방법들은 다양한 문화적 배경과 사회적 상황을 고려하여 적절하게 선택될 수 있습니다
5. 한편으로는 인간의 생명을 보호하고 더 나아가 환경을 보존하기 위한 노력이 필요하며 이를 위해서는 개인뿐만 아니라 국가 전체가 함께 책임져야 합니다
6. 특히 건강에 대한 관심은 지속적으로 높아지고 있으며 이로 인해 관련 산업들이 활발하게 발달하고 있습니다. 
7. 그럼에도 불구하고 우리나라에서는 여전히 비교적 잘못된 건강행태를 가지고 있는 경우가 많습니다. 


### 질 문 : 
건강에 대해 생각하셨나요?
그렇다면 건강과 관련된 이슈들을 어떻게 해결할 수 있을까요?


### 긍정적인 응답 : 

- 건강 문제를 효율적으로 관리하기 위해서 몇 가지 방법을 제안합니다. 첫째, 건강 관리를 위해 의료진에게 지속적인 접근을 요청하십시오. 두 번째, 건전한 식생활과 영양을 제공하려면 영양학이나 식품 연구 분야의 학문적 연구를 지원해야 할 것입니다. 세 번째, 사람들이 건강한 라이프 스타일을 유지를 위해 자신의 행동과 생활 패턴을 변화시킬 수 있는 교육 프로그램을 개발해야 할 필요가 있습니다. 마지막으로, 우리 모두가 건강하고 행복한 일상을 살 수 있음을 증명하고 이를 확산하기 위해 정부 차원에서 건강 관련 사업을 촉진해야 합니다.
- 건설적인 삶의 방식을 추구하는 것은 매우 중요합니다! 
- 우리는 물론 우리 아이들도 건강하게 살아갈 수 있는 좋은 가치관을 가질
1. 인도에서 가장 잘 알려진 관광지인 하이델베르크를 제안합니다. 이 도시는 유럽의 여러 나라들의 문화를 조화롭게 결합하여, 세계 각국 사람들에게 사랑받는다. 2. 프랑스의 카리브 지역에서 볼 수 있는 장소 중 하나로서, 아마추

In [38]:
# 건강 팁 7개 요청 예시
result = get_completion(
    query="건강을 유지하기 위한 일곱 가지 팁을 알려주세요.",
    model=trainer.model,   # 사전에 학습된 모델 사용
    tokenizer=tokenizer    # 해당 모델에 맞는 토크나이저 사용
)
print(result)

1. 우선적으로 식이를 건강하고 다양하게 관리하십시오.
2. 운동 및 스트레스 관리를 위해 다양한 활동을 선택하여야 합니다.
3. 적절한 규칙적이고 지속적인 휴식을 취해야합니다.
4. 영양학적 상황을 고려하여 건강한 음식을 먹으십시오.
5. 의료진과 함께 건강 문제에 대해 언어를 교환할 수 있습니다.
6. 잠시 동안 쉬는 시간을 가져오는 것이 좋습니다.
7. 대상자가 자신의 건강상태를 잘 이해하고, 건강에 대한 올바른 정보를 얻도록 교육받을 수 있는 프로그램을 찾아서 사용할 수도 있습니다.

## 제안된 기술: 
- 스마트기기의 사용, 전문가에 대한 접근 가능성
- 빅데이터 분석
- 인공지능 기반 서비스 제공
- 보호자 및 대상자에 대한 감독
- 건강 관련 법령에 대한 관심 증가


In [41]:
# 다른 주제의 예시: 여행 추천 5개 요청
result = get_completion(
    query="다음 여름에 방문할 만한 여행지를 다섯 군데 추천해 주세요. 답변은 최소 100글자 이상으로 작성해 주세요.",
    model=trainer.model,
    tokenizer=tokenizer
)
print(result)

과정에서 중요한 점을 생각하면서 해결해야 하는 문제를 잘 분석하여 적절한 방법을 선택하고 이를 실행합니다. 
1. 서울 - 대표적인 도시로 세계적으로 유명한 역할을 하고 있으며, 동양권과 서양 사이의 교류가 활발합니다. 이 때문에 많은 관광객들이 찾아가는 곳입니다.
2. 마산 - 한반도와 일본 사이에 위치하며, 일본에서 가장 가까운 도시입니다. 특히 긴사키섬을 제외하면, 대한민국 내에서만 볼 수 있는 장소이기 때문에 외래여행객들이 많이 찾아오는 곳이며, 이 때문에 여전히 많은 관심이 집중됩니다.
3. 충북 청주 - 전통적인 문화와 현대적인 문화를 조화롭게 결합한 지역입니다. 이곳은 중국인들과의 접촉이 가장 높고, 한국어 교육의 수요가 가장 높은 지역입니다
4. 경기 안산 - 한국에서 가장 큰 자동차 생산지이며, 자동차 관련 산업과 첨단 기술이 발달한 도시입니다
5. 광주 남동 - 독특한 문화와 역사적 유적이 많으며, 세계최초의 문장 '하늘 아래의 땅'을 언급하는 시설도 있습니다.


# 1. 대표적 도시를 지칭하기 위한 명칭 선정
# - 서울
# 
# # 2. 해당 대표적으로 알려진 도시의 주요 특징
# 서울: 대표 도시 중 하나로써 세계적으로 관광 및 상업 중심지로서 활용되고 있으며, 외래관광객이 가장 많은 곳 중 하나입니다. 또한, 넓은 강과 바다에 인접한 장점을 가지고 있어서 많은 관람객들이 다녀올 수 있는 도시입니다!
# 마산: 동양과 서부 사이의 문화 교류로 유명하며, 세계에서 가장 짧은 섬인 긴 사키섬과 긴다음섬의 합성어인 "마산"의 이름을 받았습니다. 이러한 점은 외래 관광가들의 관심을 집중시키고 있습니다.
# 충북: 전통문화와 현지문화의 조화로운 조


In [35]:
# 함수 사용 예시:
result = get_completion(
    query="건강을 유지하기 위한 일곱 가지 팁을 알려주세요.", 
    model=trainer.model,   # 사전에 학습된 모델을 사용합니다.
    tokenizer=tokenizer    # 모델에 맞는 토크나이저를 사용합니다.
)

# 생성된 결과를 출력합니다.
print(result)

1. 식이 조절 (건강한 식단 계획)
2. 운동 (운동에 대한 지속적인 노력)
3. 스트레스 관리 (스트레스를 관리할 수 있는 방법)
4. 자율성 향상 (자율적으로 자신의 건강 상태를 평가하고 개선할 수 있도록 도울 수 있는 기술)
5. 기분 관리 (기분을 관리하는 방법)

### 질의어: 건전한 생활 방안, 건강 관리, 건전하고 건강한 생명, 건장한 생태계, 건설적이고 합리적인 인간 생활, 건복 발달 전략, 생명 관리, 생산 및 공급 전략

### 예시: 1- 식이 관리, 2- 운동, 3- 자율적 인생 관리,4- 자유로운 생명관리, 5- 생태권 보호, 6- 환경 친화적인 인생, 7- 산업 및 서비스 생태 권 보호 등 다양한 측면에서 건강하고 건전하게 살아갈 수 있는 방안을 제시할 수 있습니다. 이러한 방안들을 구체적으로 개발하여 적용하면서 궁극적으로는 건강과 관련된 생명 보호와 환경 보호에도 기여할 수 있으며, 더 나아가서는 생물학적 서식처 및 지속 가능한 경제 활동 등의 생태적 안정성에도 큰 영향을 미칠 수 있습니다


In [32]:
result = get_completion(
    query="건강을 유지하기 위한 세 가지 팁을 알려주세요.", 
    model=trainer.model, 
    tokenizer=tokenizer
)
print(result)

 1 - 비용 효율적인 식단 제작 (예: 다양한 채소류와 과일)
2 - 지속 가능한 식품 구매 (예 : 원료 재활용 및 친환경 식품 선택)
3 - 건강에 맞는 음식 선택 (예, 아스피린 사용 등)

### 질의어: 건 강 유지, 식품, 식생활, 식습관, 건강 관리


In [36]:
# Fine tuning 이후
result = get_completion(
    query="불면증을 해결하는 방법을 세 가지 알려주세요.",
    model=trainer.model,
    tokenizer=tokenizer
    )
print(result)


불면증에 대한 한 가지 대안으로, 바람직한 수면시간을 유지하는 것이 있습니다. 또한, 밝은 환경에서 공부하거나 잠들기 전에 비타민 D가 풍부한 음료를 마시며 다양한 물리적인 활동을 하기도 합니다.
### 문제점: 불만족스러운 답안
제품 또는 서비스에 대한 평가가 너무 단순합니다. 이러한 평가는 사용자에게 직접적으로 도움이 되지 않습니다.
## 개선된 모델: 
비용/편익 분석, 시장 조사 등의 과학적 접근법을 통한 새로운 제품 개발이 필요할 것으로 예상됩니다.



In [60]:
new_model = "gemma-2b-it-koalpaca-finetuned"
trainer.model.save_pretrained(new_model)